In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ''
KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# Hybrid Recommender System

A recommender system, or a recommendation system, can be thought of as a subclass of information filtering system that seeks to predict the best “rating” or “preference” a user would give to an item which is typically obtained by optimizing for objectives like total clicks, total revenue, and overall sales.

Broadly speaking, most recommender systems leverage two types of data:
* Interaction Data, such as ratings, and browsing behaviors, and
* Attribution Information, about each user and items

The modeling approach relying on the former data is generally known Collaborative Filtering method.

# What Is Collaborative Filtering?

Collaborative filtering is a technique that can filter out items that a user might like on the basis of reactions by similar users. It works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user. It looks at the items they like and combines them to create a ranked list of suggestions. There are many ways to decide which users are similar and combine their choices to create a list of recommendations.


**Item-Based Collaborative Filtering** is the recommendation system to use the similarity between items using the ratings by users.

**User-Based Collaborative Filtering** is a technique used to predict the items that a user might like on the basis of ratings given to that item by the other users who have similar taste with that of the target user. Many websites use collaborative filtering for building their recommendation system.


# Business Problem

Estimate book recomendation using the item-based and user-based recommender methods for the user whose ID is given.

# Load Dataset


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coursev2/coursev2.csv
/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/user-course/user_coursev3.csv


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 20)
course = pd.read_csv("/kaggle/input/coursev2/coursev2.csv")

course.sort_values(by='id').head(05)

,id,title,user_id,rating_point
150849,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,91110.0,4.5
139086,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,113153.0,5.0
141585,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,84787.0,5.0
88146,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,92938.0,5.0
379006,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,99474.0,4.5
318285,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,3100552.0,4.0
1581,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,106773.0,4.5
89822,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,1116.0,5.0
376897,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,50597.0,4.5
207869,1,মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন...,99011.0,5.0


In [ ]:
course.shape

(932746, 4)

# Let's Start analyse the data!

In [ ]:
# Let's find the unique courses:
course["title"].nunique()

135

In [ ]:
 #find the unique users:
course["user_id"].nunique()

442382

In [ ]:
#see how many users were made on which course:
course["title"].value_counts().head()

শুধুমাত্র শিক্ষকদের জন্য (যাদের শিক্ষক আইডি আছে)    641551
Batch-1                                             203989
MyGov for Government Employees                       13135
MyGov for Citizens                                    9830
১ম ব্যাচ                                              6874
Name: title, dtype: int64

In [ ]:
# get to the courses with less than 1000 reviews:
#comment_counts = pd.DataFrame(df["title"].value_counts())
#rare_courses = comment_counts[comment_counts["title"] <= 1000].index

# get access to courses with over 1000 reviews:
#common_courses = df[~df["title"].isin(rare_courses)]
#common_courses.shape

In [ ]:
# Unique courses with more than 1000 reviews:
#commoncourses["title"].nunique()

In [ ]:
# create the User course Df:
user_course_df = course.pivot_table(index=["user_id"], columns=["title"], values="rating_point")

#courses that users have voted for.
user_course_df.shape

(439426, 122)

In [ ]:
user_course_df.head(5)

title,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৩য় ব্যাচ,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৪র্থ ব্যাচ,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৫ম ব্যাচ,B.Sc Nursing: Fundaments of Nursing-2 (Paper I),B.Sc Nursing: Newborn Nursing,BLMT-2,Batch-1,Batch-1 (April-2021),Batch-2,C-Programming (সি প্রোগ্রামিং)- ২য় ব্যাচ,...,সাইবার সিকিউরিটি কোর্স [পরীক্ষামূলক কোর্স],সৃজনশীল প্রশ্নের উত্তরপত্র মূল্যায়ন: পদার্থবিজ্ঞান (৩য় ব্যাচ),স্কুলে নৈতিকতা মূল্যবোধের শিক্ষা এবং সহানুভূতি শেখানো - ব্যাচ ১,স্বল্প পুঁজিতে কোয়েল পালন (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ১ম ব্যাচ (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ২য় ব্যাচ (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ৩য় ব্যাচ,হাউস কিপিং: গৃহকর্ম প্রশিক্ষণ (House Keeping),১ম ব্যাচ,৩৯তম বিসিএস (স্বাস্থ্য) ওরিয়েন্টশন প্রশিক্ষণ
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71.0,NaN,NaN,NaN,NaN,NaN,NaN,3.5,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
notnull_df = user_course_df.notnull()
count_series = notnull_df.sum(axis=1)
max_index = count_series.idxmax()
print("Row with most non-null values: ", max_index)
print("Count of non-null values: ", count_series[max_index])

Row with most non-null values:  3079428.0
Count of non-null values:  81


In [ ]:
# item-based course recommendation example:
course_name = "মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন্মুক্ত DCD)"
course_name = user_course_df[course_name]
user_course_df.corrwith(course_name).sort_values(ascending=False).head(10)

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
নথি (Nothi) অফিস ব্যবস্থাপনা (ই-ফাইল সিস্টেম)                                1.0
ব্যাচ- ২য়                                                                    1.0
Diploma Nursing: Leadership and Management                                   1.0
সঠিকভাবে পড়তে শেখা (Supplemantery Reading Materials Development) ২য় ব্যাচ    1.0
শ্রেণি ব্যবস্থাপনার কৌশল (৫ম ব্যাচ)                                          1.0
মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন্মুক্ত DCD)                    1.0
ব্যাচ-২য়                                                                     1.0
ধানের ব্লাস্ট রোগ-৩য় ব্যাচ                                                   1.0
ব্যাচ- ৩য়                                                                    1.0
ব্যাচ - ১                                                                    1.0
dtype: float64

In [ ]:
#determine the courses that the user watched.

# choose random user:
#random_user = int(pd.Series(user_course_df.index).sample(1, random_state=45).values)
#random_user = 23593
random_user = 3079428


In [ ]:
#reduce the dataset to according to user:
random_user_df = user_course_df[user_course_df.index == random_user]
random_user_df

title,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৩য় ব্যাচ,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৪র্থ ব্যাচ,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৫ম ব্যাচ,B.Sc Nursing: Fundaments of Nursing-2 (Paper I),B.Sc Nursing: Newborn Nursing,BLMT-2,Batch-1,Batch-1 (April-2021),Batch-2,C-Programming (সি প্রোগ্রামিং)- ২য় ব্যাচ,...,সাইবার সিকিউরিটি কোর্স [পরীক্ষামূলক কোর্স],সৃজনশীল প্রশ্নের উত্তরপত্র মূল্যায়ন: পদার্থবিজ্ঞান (৩য় ব্যাচ),স্কুলে নৈতিকতা মূল্যবোধের শিক্ষা এবং সহানুভূতি শেখানো - ব্যাচ ১,স্বল্প পুঁজিতে কোয়েল পালন (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ১ম ব্যাচ (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ২য় ব্যাচ (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ৩য় ব্যাচ,হাউস কিপিং: গৃহকর্ম প্রশিক্ষণ (House Keeping),১ম ব্যাচ,৩৯তম বিসিএস (স্বাস্থ্য) ওরিয়েন্টশন প্রশিক্ষণ
user_id,,,,,,,,,,,,,,,,,,,,,
3079428.0,NaN,5.0,5.0,5.0,5.0,NaN,4.975,5.0,NaN,5.0,...,5.0,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [ ]:
#choose non-NaN. courses watched by all 28491:
course_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
len(course_watched)

81

In [ ]:
#verify dataframe:
user_course_df.loc[user_course_df.index == random_user, user_course_df.columns == "মাল্টিমিডিয়া ক্লাসরুম মনিটরিং এন্ড মেন্টরিং (১ম ব্যাচ)"]
# gave this course rating.

title,মাল্টিমিডিয়া ক্লাসরুম মনিটরিং এন্ড মেন্টরিং (১ম ব্যাচ)
user_id,
3079428.0,5.0


In [ ]:
#How many courses have user watched:
len(course_watched)

81

In [ ]:
#reduce the dataset based on courses watched by user 28491:
course_watched_df = user_course_df[course_watched]
course_watched_df.head()
#course_watched_df.shape

title,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৪র্থ ব্যাচ,(BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৫ম ব্যাচ,B.Sc Nursing: Fundaments of Nursing-2 (Paper I),B.Sc Nursing: Newborn Nursing,Batch-1,Batch-1 (April-2021),C-Programming (সি প্রোগ্রামিং)- ২য় ব্যাচ,Diploma Nursing: Anatomy and Physiology,Diploma Nursing: Leadership and Management,MMCD-মাল্টিমিডিয়া কনটেন্ট তৈরি (সকলের জন্য উন্মুক্ত)- ২য় ব্যাচ,...,সহজ ইংরেজি লেখার কৌশল -২য় ব্যাচ,সাংবাদিকদের জন্য কোভিড-১৯ বিষয়ক প্রশিক্ষণ বাংলা কোর্স,সাইবার সিকিউরিটি কোর্স [পরীক্ষামূলক কোর্স],স্বল্প পুঁজিতে কোয়েল পালন (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ১ম ব্যাচ (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ২য় ব্যাচ (Quail Farming in Low Budget),স্বল্প পুঁজিতে কোয়েল পালন: ৩য় ব্যাচ,হাউস কিপিং: গৃহকর্ম প্রশিক্ষণ (House Keeping),১ম ব্যাচ,৩৯তম বিসিএস (স্বাস্থ্য) ওরিয়েন্টশন প্রশিক্ষণ
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#information on how many courses each user watched in total:
user_course_count = course_watched_df.T.notnull().sum()

user_course_count = user_course_count.reset_index()
user_course_count.columns = ["user_id","course_count"]
user_course_count.head()

,user_id,course_count
0,1.0,3
1,22.0,1
2,23.0,1
3,25.0,1
4,67.0,1


In [ ]:
len(user_course_count)
#user_course_count.shape

439426

In [ ]:
# user watched courses:
#user_course_count[user_course_count["course_count"] >= 3].count()

In [ ]:
# 40% of courses watched by 28491:
perc = len(course_watched) * 40 / 100
perc

32.4

In [ ]:
# People who have watched more than 3 courses together with user:
users_same_course = user_course_count[user_course_count["course_count"] >= 3]["user_id"]
print(users_same_course.count())
print(users_same_course.head())

15116
0       1.0
5      68.0
6      71.0
12     91.0
22    123.0
Name: user_id, dtype: float64


In [ ]:
#combine the data of the user and similar users:
final_df = pd.concat([course_watched_df[course_watched_df.index.isin(users_same_course)],
                      random_user_df[course_watched]])

print(final_df.shape)
print(final_df.head())
final_df.T.corr()


(15117, 81)
title    (BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৪র্থ ব্যাচ  \
user_id                                                  
1.0                                                NaN   
68.0                                               NaN   
71.0                                               NaN   
91.0                                               NaN   
123.0                                              NaN   

title    (BTT) বেসিক টিচার্স ট্রেনিং কোর্স- ৫ম ব্যাচ  \
user_id                                                
1.0                                              NaN   
68.0                                             NaN   
71.0                                             NaN   
91.0                                             NaN   
123.0                                            NaN   

title    B.Sc Nursing: Fundaments of Nursing-2 (Paper I)  \
user_id                                                    
1.0                                                  NaN   
68.0    

user_id,1.0,68.0,71.0,91.0,123.0,159.0,218.0,239.0,251.0,294.0,...,4568599.0,4568728.0,4568884.0,4569515.0,4570490.0,4570805.0,4575709.0,4585366.0,4590145.0,3079428.0
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000
68.0,NaN,1.000000,-1.000000,-1.000000,NaN,NaN,1.000000,NaN,NaN,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.989373
71.0,NaN,-1.000000,1.000000,1.000000,NaN,NaN,-1.000000,-1.0,NaN,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.948718
91.0,NaN,-1.000000,1.000000,1.000000,NaN,NaN,-1.000000,NaN,NaN,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.755929
123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570805.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4575709.0,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.500000
4585366.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#corr for all users:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
print(corr_df.head())

corr_df = pd.DataFrame(corr_df, columns=["corr"]).drop_duplicates()
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

user_id    user_id  
162861.0   2926.0      -1.0
112393.0   3706158.0   -1.0
3234814.0  3706158.0   -1.0
3706158.0  2976481.0   -1.0
3022.0     3706158.0   -1.0
dtype: float64


,user_id_1,user_id_2,corr
0,162861.0,2926.0,-1.0
1,112393.0,3706158.0,-1.0
2,3234814.0,3706158.0,-1.0
3,3706158.0,2976481.0,-1.0
4,3022.0,3706158.0,-1.0


In [ ]:
# Users with a correlation of %40 or more with the user:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.4)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "user_id"}, inplace=True)
print(top_users.head())
top_users.shape

       user_id      corr
630  3708402.0  0.999613
629  3239888.0  0.999495
628  3175096.0  0.999440
627  3265707.0  0.999015
626  3315039.0  0.998853


(631, 2)

In [ ]:
print(top_users.columns)

Index(['user_id', 'corr'], dtype='object')


In [ ]:
# let's see the ratings of users:
top_users_ratings = top_users.merge(course[["user_id","id", "rating_point"]], how='inner')

top_users_ratings = top_users_ratings[top_users_ratings["user_id"] != random_user]
print(top_users_ratings.head())
top_users_ratings.shape

     user_id      corr   id  rating_point
0  3708402.0  0.999613  898           5.0
1  3708402.0  0.999613  848           5.0
2  3708402.0  0.999613  656           5.0
3  3708402.0  0.999613  200           5.0
4  3708402.0  0.999613  861           5.0


(9237, 4)

In [ ]:
# Calculate the Weighted Average Recommendation Score and keep the first 5 courses.

#single score with the most similar by corr * rating:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating_point']
top_users_ratings.groupby('id').agg({"weighted_rating": "mean"})

recommendation_df = top_users_ratings.groupby('id').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
print(recommendation_df.head())
recommendation_df.shape

   id  weighted_rating
0   5         3.916059
1  10         2.000845
2  11         3.419103
3  12         4.841648
4  18         3.343714


(170, 2)

In [ ]:
# weighted rating greater than 4:
recommendation_df[recommendation_df["weighted_rating"] > 3.7]

# courses user will like:
#course_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.7].sort_values("weighted_rating", ascending=False)
course_to_be_recommend = recommendation_df["weighted_rating"] > 3.7
course_to_be_recommend = recommendation_df.sort_values("weighted_rating", ascending=False)
course_to_be_recommend=course_to_be_recommend.merge(course[["id", "title"]],how='inner')

# Drop duplicate courses based on their "id" value
course_to_be_recommend = course_to_be_recommend.drop_duplicates(subset="id")


#Let's see the top 5 courses:
course_to_be_recommend.shape


(170, 3)

In [ ]:
# Item-based suggestion based on the name of the course that the user has watched with the highest score.

# ▪ 5 suggestions user-based
# ▪ 5 suggestions item-based

course = pd.read_csv("/kaggle/input/coursev2/coursev2.csv")
#print(course.head())

# The last highly-rated course by user:
#user = 65578
user = 3079428
course_id = course[(course["user_id"] == user) & (course["rating_point"] == 5.0)].sort_values(by="rating_point", ascending=False)["id"][0:1].values[0]
course_id

299

In [ ]:
# ▪ 5 suggestions user-based
course_to_be_recommend[:5]['title'].to_list()

['সহজ ইংরেজি লেখার কৌশল -২য় ব্যাচ',
 'পুকুরে মিশ্র মাছ চাষের কৌশল- ২য় ব্যাচ',
 'ট্রাবলশ্যুটিং ২: ইন্টারনেট সংযোগ সংক্রান্ত (৩য় ব্যাচ)',
 'সঠিকভাবে পড়তে শেখা (Supplemantery Reading Materials Development)',
 'ট্রাবলশ্যুটিং ৫: অপারেটিং সিস্টেম',
 'পড়তে শেখার নির্দেশনা (Reading Instruction) -সকলের জন্য উন্মুক্ত- ২য় ব্যাচ',
 'Batch-1',
 'ট্রাবলশ্যুটিং ৩: কম্পিউটার নিরাপত্তা (২য় ব্যাচ)',
 'শিক্ষক : ই-লার্নিং প্ল্যাটফর্মের ভিডিও টিউটোরিয়াল',
 'ট্রাবলশ্যুটিং ১: মাল্টিমিডিয়া প্রজেক্টর (৫ম ব্যাচ)',
 'সাংবাদিকদের জন্য কোভিড-১৯ বিষয়ক প্রশিক্ষণ বাংলা কোর্স',
 'শুধুমাত্র শিক্ষকদের জন্য (যাদের শিক্ষক আইডি আছে)',
 'স্বল্প পুঁজিতে কোয়েল পালন (Quail Farming in Low Budget)',
 'Batch-1',
 'Negotiation Techniques For Fresh Graduates']

In [ ]:
# ▪ 5 suggestions item-based
course_name = course[course['id'] == course_id]['title'].values[0]
course_name = user_course_df[course_name]
course_from_item_based = user_course_df.corrwith(course_name).sort_values(ascending=False)
course_from_item_based[1:6].index.to_list()

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


['Effective Teaching- Batch 1',
 'ব্যাচ ১',
 'সমন্বয়কারী : ই-লার্নিং প্ল্যাটফর্মের ভিডিও টিউটোরিয়াল',
 'C-Programming (সি প্রোগ্রামিং)- ২য় ব্যাচ',
 'ট্রাবলশ্যুটিং ১: মাল্টিমিডিয়া প্রজেক্টর (২য় ব্যাচ)',
 'সাইবার সিকিউরিটি কোর্স [পরীক্ষামূলক কোর্স]',
 'পড়তে শেখার নির্দেশনা (Reading Instruction) -সকলের জন্য উন্মুক্ত- ৩য় ব্যাচ',
 'Comprehensive Training Course on COVID-19 for Doctors - English',
 'প্রফেশনাল ভিডিও টিউটোরিয়াল ডেভেলপমেন্ট- ৩য় ব্যাচ',
 'প্রফেশনাল ভিডিও টিউটোরিয়াল ডেভেলপমেন্ট- ৪র্থ ব্যাচ',
 'ট্রাবলশ্যুটিং ২: ইন্টারনেট সংযোগ সংক্রান্ত (২য় ব্যাচ)',
 'ট্রাবলশ্যুটিং ৫: অপারেটিং সিস্টেম',
 'সমস্যা সমাধানভিত্তিক শিখন ও গাঠনিক মূল্যায়ন (২য় ব্যাচ) - Problem-based Learning and Formative Assessment (2nd Batch)',
 'মাল্টিমিডিয়া কনটেন্ট তৈরি -MMCD (সকলের জন্য উন্মুক্ত DCD)',
 'Research Methodology for Social Science Researchers']